<a name="top"> <h1>02. Neuronal Networks</h1> <a>

<p>Análisis de sentimiento: Tweets<br />
<strong>Trabajo de Fin de Master</strong><br />
<strong>Master Universitario en Ciencia de Datos</strong></p>

<p>&nbsp;</p>

<p style="text-align:right">V&iacute;ctor Viloria V&aacute;zquez (<em>victor.viloria@cunef.edu</em>)</p>


<hr style="border:1px solid gray">

### Estructura

[1. Librerias utilizadas y funciones](#librerias) 

[2. Introducción ](#introduccion) 

   - Objetivo de negocio.

[3. Yelp Dataset ](#yelp) 

   - Información del dataset
   - Características del dataset


[4. Transformación del formato de ficheros](#transformacion) 


[5. Transformación de datos](#datos)

   - Business
       - Carga del fichero
       - Transformación de los datos
       - Exportación de ficheros procesados

<hr style="border:1px solid gray">

# <a name="librerias"> 1. Librerias utilizadas y funciones <a>


Importamos las librerias a utilizar para el preprocesamiento:

In [8]:
# Import libraries.

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
import numpy as np
import string

# Import neural network libraries.

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Dropout, Dense

# <a name="lectura"> 2. Lectura del dataframe <a>


In [9]:
#Import parquet file.

reviews = pd.read_parquet('../../data/processed/tweets.parquet')

# Show the head of the dataframe.

reviews.head()

,text,sentiment,SentimentText_clean
0,id have responded if i were going,0,id responded going
1,sooo sad i will miss you here in san diego,2,sooo sad miss san diego
2,my boss is bullying me,2,boss bullying
3,what interview leave me alone,2,interview leave alone
4,sons of why couldnt they put them on the rel...,2,sons couldnt put releases already bought


In [10]:
tweets = reviews["SentimentText_clean"].tolist()
sentimientos = reviews["sentiment"].values

In [11]:
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(tweets)
word_index = tokenizer.word_index

In [12]:
sequences = tokenizer.texts_to_sequences(tweets)


In [13]:
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding="post", truncating="post")


In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(word_index) + 1, output_dim=64, input_length=max_length),
    tf.keras.layers.Conv1D(128, 5, activation="relu"),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(3, activation="softmax")
])


In [15]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Entrenar el modelo
model.fit(padded_sequences, sentimientos, epochs=10, validation_split=0.2)

Epoch 1/10
687/687 [==============================] - 9s 12ms/step - loss: 0.7954 - accuracy: 0.6446 - val_loss: 0.6849 - val_accuracy: 0.7164
Epoch 2/10
687/687 [==============================] - 8s 12ms/step - loss: 0.5443 - accuracy: 0.7835 - val_loss: 0.7268 - val_accuracy: 0.7047
Epoch 3/10
687/687 [==============================] - 8s 12ms/step - loss: 0.3422 - accuracy: 0.8779 - val_loss: 0.8481 - val_accuracy: 0.6876
Epoch 4/10
687/687 [==============================] - 8s 12ms/step - loss: 0.1872 - accuracy: 0.9393 - val_loss: 1.0929 - val_accuracy: 0.6678
Epoch 5/10
687/687 [==============================] - 8s 12ms/step - loss: 0.1020 - accuracy: 0.9690 - val_loss: 1.2762 - val_accuracy: 0.6627
Epoch 6/10
687/687 [==============================] - 12s 17ms/step - loss: 0.0644 - accuracy: 0.9811 - val_loss: 1.5388 - val_accuracy: 0.6689
Epoch 7/10
687/687 [==============================] - 12s 18ms/step - loss: 0.0460 - accuracy: 0.9863 - val_loss: 1.7358 - val_accuracy: 0.65

In [24]:
# Crear el modelo de la red neuronal
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(word_index) + 1, output_dim=64, input_length=max_length),
    tf.keras.layers.Conv1D(128, 5, activation="relu"),
    tf.keras.layers.MaxPooling1D(5),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv1D(64, 5, activation="relu"),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation="softmax")
])

# Compilar el modelo
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Entrenar el modelo con validación cruzada
history = model.fit(padded_sequences, sentimientos, epochs=20, validation_split=0.2, batch_size=32)



Epoch 1/20
687/687 [==============================] - 22s 29ms/step - loss: 1.0151 - accuracy: 0.4689 - val_loss: 0.7697 - val_accuracy: 0.6631
Epoch 2/20
687/687 [==============================] - 5s 7ms/step - loss: 0.7289 - accuracy: 0.6988 - val_loss: 0.6758 - val_accuracy: 0.7178
Epoch 3/20
687/687 [==============================] - 4s 6ms/step - loss: 0.5802 - accuracy: 0.7719 - val_loss: 0.7030 - val_accuracy: 0.7142
Epoch 4/20
687/687 [==============================] - 4s 7ms/step - loss: 0.4708 - accuracy: 0.8212 - val_loss: 0.7474 - val_accuracy: 0.7033
Epoch 5/20
687/687 [==============================] - 4s 6ms/step - loss: 0.3847 - accuracy: 0.8555 - val_loss: 0.8447 - val_accuracy: 0.7022
Epoch 6/20
687/687 [==============================] - 3s 5ms/step - loss: 0.3166 - accuracy: 0.8860 - val_loss: 0.9472 - val_accuracy: 0.6875
Epoch 7/20
687/687 [==============================] - 5s 7ms/step - loss: 0.2697 - accuracy: 0.9041 - val_loss: 1.0619 - val_accuracy: 0.6789
Epoc

In [28]:
# Crear el modelo de la red neuronal
model = tf.keras.Sequential([
    Embedding(input_dim=len(word_index) + 1, output_dim=64, input_length=max_length),
    MaxPooling1D(5),
    Dropout(0.5),
    Conv1D(64, 5, activation="relu"),
    GlobalMaxPooling1D(),
    Dropout(0.5),
    Dense(64, activation="relu"),
    Dropout(0.5),
    Dense(3, activation="softmax")
])

# Compilar el modelo
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Entrenar el modelo con validación cruzada
history = model.fit(padded_sequences, sentimientos, epochs=20, validation_split=0.2, batch_size=32)


Epoch 1/20
687/687 [==============================] - 8s 12ms/step - loss: 1.0202 - accuracy: 0.4670 - val_loss: 0.8594 - val_accuracy: 0.5701
Epoch 2/20
687/687 [==============================] - 8s 12ms/step - loss: 0.8210 - accuracy: 0.6325 - val_loss: 0.7230 - val_accuracy: 0.7004
Epoch 3/20
687/687 [==============================] - 8s 12ms/step - loss: 0.6926 - accuracy: 0.7123 - val_loss: 0.6940 - val_accuracy: 0.7097
Epoch 4/20
687/687 [==============================] - 8s 11ms/step - loss: 0.6116 - accuracy: 0.7555 - val_loss: 0.7083 - val_accuracy: 0.6911
Epoch 5/20
687/687 [==============================] - 8s 12ms/step - loss: 0.5469 - accuracy: 0.7890 - val_loss: 0.7112 - val_accuracy: 0.7097
Epoch 6/20
687/687 [==============================] - 8s 12ms/step - loss: 0.5012 - accuracy: 0.8103 - val_loss: 0.7398 - val_accuracy: 0.7011
Epoch 7/20
687/687 [==============================] - 8s 12ms/step - loss: 0.4609 - accuracy: 0.8271 - val_loss: 0.7676 - val_accuracy: 0.6957

In [18]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

In [30]:
model.save("../../models/nn_tweets.h5")